In [1]:
# from pose_estimator_2d import openpose_estimator
# from pose_estimator_3d import estimator_3d
from utils import smooth, vis, camera
from bvh_skeleton import openpose_skeleton, h36m_skeleton, cmu_skeleton

import cv2
import importlib
import numpy as np
import os
from pathlib import Path
from IPython.display import HTML
import math
import mathutils

In [2]:
mapping = [(0, None), (4, None), (1, None), (7, None), (5, None), (2, None), (3, None), (6, None), (12, None), (8, None), (15, None), (None, [0, 0, 0]), (9, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (10, None), (13, None), (11, None), (14, None), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0]), (None, [0, 0, 0])]

In [3]:
# Create function which takes X3D file as input and outputs SMPL pose params and translation

## Whole Dataset

In [4]:
h36m_skel = h36m_skeleton.H36mSkeleton()

In [5]:
def euler_to_quaternion_zxy(euler_z, euler_x, euler_y):
    # Convert degrees to radians
    euler_z = math.radians(euler_z)
    euler_x = math.radians(euler_x)
    euler_y = math.radians(euler_y)

    # Create Euler object with 'ZXY' rotation order
    euler = mathutils.Euler((euler_z, euler_x, euler_y))

    # Convert to quaternion
    quaternion = euler.to_quaternion()

    # Return as a tuple in the desired order (W, Y, Z, X)
    return (quaternion.w, quaternion.y, quaternion.z, quaternion.x)


def rodrigues_from_pose(frame, joint_name, all_rotations):

    quat = all_rotations[frame][joint_name]
    quat = mathutils.Quaternion(quat)
    (axis, angle) = quat.to_axis_angle()
    rodrigues = axis
    rodrigues.normalize()
    rodrigues = rodrigues * angle
    return rodrigues


def apply_mapping(B, mapping, group_size=3):
    """
    Apply a mapping to reorder and pad a list like B.
    """
    # Divide B into groups of group_size
    grouped_B = [B[i:i + group_size] for i in range(0, len(B), group_size)]

    # Apply mapping
    output_groups = []
    for index, padding in mapping:
        if index is not None:
            output_groups.append(grouped_B[index])  # Reorder from B
        elif padding is not None:
            output_groups.append(padding)  # Add padding group

    # Flatten the output groups
    reordered_B = [item for group in output_groups for item in group]
    return reordered_B


def load_and_prepare_data(data):

    # Load the 3D pose data from the .npy file
    pose_data = data

    # Get the number of frames and joints
    n_frames, n_joints, _ = pose_data.shape

    # Define the rotation matrix for -90 degrees around the x-axis
    rotation_matrix = np.array([
        [1,  0,  0],
        [0,  0, 1],
        [0,  -1,  0]
    ])

    # Apply the rotation matrix to every frame and every joint
    # We reshape the pose_data to (n_frames * n_joints, 3), apply the rotation, then reshape back
    data = np.dot(pose_data.reshape(-1, 3), rotation_matrix.T).reshape(n_frames, n_joints, 3)


    return data

def calculate_translations(channels):

    all_translations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        # Iterate through the list and extract translation values
        translation_values = channels[index][0:3]  # Extracting the translation values

        all_translations.append(translation_values)

    return all_translations


def calculate_rotations(channels, body_parts):

    all_rotations = []

    for index in range(len(channels)):
        # Initialize the dictionary to hold the results
        rotation_dict = {}
        # Iterate through the list and extract rotation values
        rotation_values = channels[index][3:]  # Extracting the rotation values, skipping the first two
        if index == 0:
            print(rotation_values)
        for i, body_part in enumerate(body_parts):
            # Take 3 consecutive values (x, y, z rotations) for each body part
            quat = euler_to_quaternion_zxy(-rotation_values[i*3+2], rotation_values[i*3+1], rotation_values[i*3])
            rotation_dict[body_part] = quat

        all_rotations.append(rotation_dict)

    return all_rotations

def calculate_all_poses(all_rotations, body_parts):

    joint_names = body_parts
    num_joints = len(joint_names)
    frame = 0
    all_poses = []


    for frame in range(len(all_rotations)):
        # Get armature pose in rodrigues representation
        pose = [0.0] * (num_joints * 3)

        for index in range(num_joints):
            joint_name = joint_names[index]
            joint_pose = rodrigues_from_pose(frame, joint_name, all_rotations)
            pose[index*3 + 0] = joint_pose[0]
            pose[index*3 + 1] = joint_pose[1]
            pose[index*3 + 2] = joint_pose[2]
        
        all_poses.append(pose)

    return all_poses

def calculate_all_thetas(all_poses):
    all_thetas = []

    for i, frame in enumerate(all_poses):
        theta_calculated = all_poses[i]

        reordered_theta = apply_mapping(theta_calculated, mapping)
        smpl_h_params = reordered_theta

        reordered_theta = reordered_theta[0:72]

        all_thetas.append(reordered_theta)
    return all_thetas

    #np.save("all_thetas_backpack.npy", all_thetas)

def save_smpl(translations, thetas, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    np.save(os.path.join(output_folder, 'all_translations.npy'), translations)
    np.save(os.path.join(output_folder, 'all_thetas.npy'), thetas)


In [6]:
def calculate_smpl_output(data):

    data = load_and_prepare_data(data)

    channels, header = h36m_skel.poses2bvh(data, output_file=None)

    all_translations = calculate_translations(channels)

    # List of body parts in the order matching the data
    body_parts = [
        'Hip', 'RightHip', 'RightKnee', 'RightAnkle', 'LeftHip', 'LeftKnee', 'LeftAnkle', 
        'Spine', 'Thorax', 'Neck', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 
        'RightShoulder', 'RightElbow', 'RightWrist'
    ]

    all_rotations = calculate_rotations(channels, body_parts)

    all_poses = calculate_all_poses(all_rotations, body_parts)


    all_thetas = calculate_all_thetas(all_poses)

    return all_translations, all_thetas

In [7]:
data = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\results\courtyard_backpack_00\X3D.npy")

In [8]:
all_thetas = calculate_smpl_output(data)

[-5.02913516932482, 9.592779420484428, 0.26763725950101497, -5.834822581234851, 5.9776781673325985, -0.5845983547809256, 0.7378565149487745, 7.27986286744448, 0.5079760299351388, 0.0, 0.0, 0.0, 6.096983266449101, 8.150579718124051, 5.711250307676071, 0.5531632663827586, 8.691498233534011, -1.6971017917687945, 0.0, 0.0, 4.969616689786745e-17, 0.2372301222093141, 6.423698643165342, 1.6858209481565583, 7.214335645018436, -5.771987337711202e-15, 2.6427798014941203e-16, 27.270152968827553, 13.626506551760764, -1.4396521229363464, -1.4244267486812954, 14.555878889620914, 87.2895521411193, -18.737606262745587, -1.2857197909239077e-15, -6.860710156890454e-15, 0.0, 0.0, 1.6896696745274934e-15, 29.891110823492518, 13.806045985880496, -86.83135500105367, 27.710357092381813, 3.1087344990751394e-15, -7.318491670104018e-15, 0.0, 3.1805546814635168e-15, 0.0]


In [9]:
def process_motionbert_files(root_path):
    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if file == 'X3D.npy':
                # Full path to the data file
                data_path = os.path.join(dirpath, file)

                # Call calculate_smpl_output
                data = np.load(data_path)
                translations, thetas = calculate_smpl_output(data)

                # Save outputs in the same directory as the data file
                output_folder = dirpath  # Save in the same directory
                save_smpl(translations, thetas, output_folder)

# Root folder containing all test sets
root_folder = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set"

In [10]:
process_motionbert_files(root_folder)

[9.987085020024338, 8.444642580752003, -3.2443970568189977, -3.372142978531094, 3.0902428499109345, -1.3888181751971829, 0.5625396561767507, 6.024957462934093, -3.5709929591574814, -7.951386703658792e-16, 0.0, 0.0, 2.631505116190481, 4.145205359534967, 6.470722649028908, -0.4108306051570296, 6.988287347824842, 0.1284670190748407, -1.1034765745125397e-32, -3.1805546814635168e-15, -3.975693351829396e-16, -0.1619396543647446, 4.539815856392692, 0.8622325320574875, -11.492228505625327, -1.0581232613368261e-15, -1.5642205751986054e-16, -0.07168130745333683, 4.0214260529329895, 3.632810891162606, -19.079987694698435, 12.173991985400347, 87.05507060292493, -19.126376396344966, 1.1058253353682686e-15, 3.0085112241297947e-15, 0.0, 0.0, 0.0, 21.146694547237285, 11.66987096675034, -87.57498452035469, 13.266052028174917, -1.085989717821699e-14, 9.868218841969103e-15, -7.951386703658792e-16, 2.2069531490250793e-32, -3.1805546814635168e-15]
[9.987085020024338, 8.444642580752003, -3.2443970568189977,

In [ ]:
#CHECK THETA SHAPE

In [11]:
import numpy as np

In [13]:
thetas = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\Alphapose\AlphaPose\examples\res\test_set\downtown_bus_00\MotionBert\person_2\all_thetas.npy")

In [14]:
thetas.shape

(2178, 72)

## Estimate 2D pose from video

In [3]:
# output = 'miscs/h36m_cxk.bvh'
# h36m_skel = h36m_skeleton.H36mSkeleton()
# _ = h36m_skel.poses2bvh(pose3d_world, output_file=output)

In [4]:
import numpy as np

In [5]:
data = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\results\X3D_interpolated.npy")

In [6]:
data = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\results\courtyard_backpack_00\X3D.npy")

In [7]:
# Load the 3D pose data from the .npy file
pose_data = data

# Get the number of frames and joints
n_frames, n_joints, _ = pose_data.shape

# Define the rotation matrix for -90 degrees around the x-axis
rotation_matrix = np.array([
    [1,  0,  0],
    [0,  0, 1],
    [0,  -1,  0]
])

# Apply the rotation matrix to every frame and every joint
# We reshape the pose_data to (n_frames * n_joints, 3), apply the rotation, then reshape back
pose_data_rotated = np.dot(pose_data.reshape(-1, 3), rotation_matrix.T).reshape(n_frames, n_joints, 3)

In [8]:
data = pose_data_rotated

In [9]:
data.shape

(1269, 17, 3)

In [10]:
# import numpy as np

# # Load the data
# data = np.load(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\results\X3D_interpolated.npy")

# # Define the rotation matrix for +90 degrees around the y-axis
# rotation_matrix_y = np.array([
#     [0, 0, 1],
#     [0, 1, 0],
#     [-1, 0, 0]
# ])

# # Apply the rotation to each keypoint in each frame
# # The shape of data is (frames, nr_keypoints, 3), so we reshape it for matrix multiplication
# data = np.einsum('ijk,kl->ijl', data, rotation_matrix_y)

# # Save the rotated data if needed
# # np.save(r"path_to_save_rotated_data.npy", rotated_data)

# print(data)


In [11]:
data.shape

(1269, 17, 3)

In [12]:
import math
import mathutils

def euler_to_quaternion_zxy(euler_z, euler_x, euler_y):
    # Convert degrees to radians
    euler_z = math.radians(euler_z)
    euler_x = math.radians(euler_x)
    euler_y = math.radians(euler_y)

    # Create Euler object with 'ZXY' rotation order
    euler = mathutils.Euler((euler_z, euler_x, euler_y))

    # Convert to quaternion
    quaternion = euler.to_quaternion()

    # Return as a tuple in the desired order (W, Y, Z, X)
    return (quaternion.w, quaternion.y, quaternion.z, quaternion.x)

# Example usage with your input
euler_x = -89.41630704972162
euler_y = 31.447267659725966
euler_z = 93.47180438041381

euler_x = -89.41630704972162
euler_y = 31.447267659725966
euler_z = 93.47180438041381

quaternion = euler_to_quaternion_zxy(-euler_z, euler_y, euler_x)
print(quaternion)

(0.607688307762146, 0.625116765499115, -0.32385730743408203, -0.36750558018684387)


In [13]:
output = 'miscs/h36m_cxk_backpack.bvh'
h36m_skel = h36m_skeleton.H36mSkeleton()

In [14]:
channels, header = h36m_skel.poses2bvh(data, output_file=output)

In [15]:
# import numpy as np

# # List of body parts in the order matching the data
# body_parts = [
#     'Hip', 'RightHip', 'RightKnee', 'RightAnkle', 'LeftHip', 'LeftKnee', 'LeftAnkle', 
#     'Spine', 'Thorax', 'Neck', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 
#     'RightShoulder', 'RightElbow', 'RightWrist'
# ]

# all_rotations = []
# for index in range(len(channels)):
#     # Initialize the dictionary to hold the results
#     rotation_dict = {}
#     # Iterate through the list and extract rotation values
#     rotation_values = channels[0][2:]  # Extracting the rotation values, skipping the first two
#     for i, body_part in enumerate(body_parts):
#         # Take 3 consecutive values (x, y, z rotations) for each body part
#         quat = euler_to_quaternion_zxy(-rotation_values[i*3+3], rotation_values[i*3+1], rotation_values[i*3])
#         rotation_dict[body_part] = quat

#     # Display the resulting dictionary
#     for body_part, rotation in rotation_dict.items():
#         print(f"{body_part}: {rotation}")


In [16]:
all_translations = []

for index in range(len(channels)):
    # Initialize the dictionary to hold the results
    translation = []
    # Iterate through the list and extract translation values
    translation_values = channels[index][0:3]  # Extracting the translation values

    all_translations.append(translation_values)

In [17]:
np.asarray(all_translations).shape

(1269, 3)

In [18]:
# Position of pelvis x,z,y
all_translations

[[0.1386837363243103, 0.0, 0.0006967457011342049],
 [0.13773773610591888, 0.0005291784182190895, 0.0031907102093100548],
 [0.13679610192775726, 0.0006080649327486753, 0.006016620434820652],
 [0.13609068095684052, 0.0005835501942783594, 0.009174376726150513],
 [0.13560856878757477, 0.00046403659507632256, 0.011572739109396935],
 [0.13487586379051208, 0.0004462634678930044, 0.014460530132055283],
 [0.13464605808258057, 0.00047239847481250763, 0.016764461994171143],
 [0.13439062237739563, 0.00048628938384354115, 0.01907649263739586],
 [0.1343507319688797, 0.0002751019783318043, 0.02049107477068901],
 [0.13464707136154175, 0.0002678155433386564, 0.02202777937054634],
 [0.13516147434711456, 0.00019240682013332844, 0.023413684219121933],
 [0.13557210564613342, 8.651800453662872e-05, 0.024756118655204773],
 [0.13592195510864258, -0.0003562995698302984, 0.025239339098334312],
 [0.13650980591773987, -0.00047283456660807133, 0.02617425099015236],
 [0.13702696561813354, -0.000652828486636281, 0.0

In [19]:
np.save("translations_backpack.npy", np.asarray(all_translations))

In [20]:
import numpy as np

# List of body parts in the order matching the data
body_parts = [
    'Hip', 'RightHip', 'RightKnee', 'RightAnkle', 'LeftHip', 'LeftKnee', 'LeftAnkle', 
    'Spine', 'Thorax', 'Neck', 'LeftShoulder', 'LeftElbow', 'LeftWrist', 
    'RightShoulder', 'RightElbow', 'RightWrist'
]

all_rotations = []
for index in range(len(channels)):
    # Initialize the dictionary to hold the results
    rotation_dict = {}
    # Iterate through the list and extract rotation values
    rotation_values = channels[index][3:]  # Extracting the rotation values, skipping the first two
    if index == 0:
        print(rotation_values)
    for i, body_part in enumerate(body_parts):
        # Take 3 consecutive values (x, y, z rotations) for each body part
        quat = euler_to_quaternion_zxy(-rotation_values[i*3+2], rotation_values[i*3+1], rotation_values[i*3])
        rotation_dict[body_part] = quat

    all_rotations.append(rotation_dict)


[-5.02913516932482, 9.592779420484428, 0.26763725950101497, -5.834822581234851, 5.9776781673325985, -0.5845983547809256, 0.7378565149487745, 7.27986286744448, 0.5079760299351388, 0.0, 0.0, 0.0, 6.096983266449101, 8.150579718124051, 5.711250307676071, 0.5531632663827586, 8.691498233534011, -1.6971017917687945, 0.0, 0.0, 4.969616689786745e-17, 0.2372301222093141, 6.423698643165342, 1.6858209481565583, 7.214335645018436, -5.771987337711202e-15, 2.6427798014941203e-16, 27.270152968827553, 13.626506551760764, -1.4396521229363464, -1.4244267486812954, 14.555878889620914, 87.2895521411193, -18.737606262745587, -1.2857197909239077e-15, -6.860710156890454e-15, 0.0, 0.0, 1.6896696745274934e-15, 29.891110823492518, 13.806045985880496, -86.83135500105367, 27.710357092381813, 3.1087344990751394e-15, -7.318491670104018e-15, 0.0, 3.1805546814635168e-15, 0.0]


In [20]:
all_rotations[0]

{'Hip': (0.99554443359375,
  0.08363642543554306,
  -0.04352453723549843,
  0.001343312906101346),
 'RightHip': (0.9973188638687134,
  0.05181387439370155,
  -0.05109219625592232,
  0.007741796318441629),
 'RightKnee': (0.9979504346847534,
  0.06345564126968384,
  0.0067073386162519455,
  -0.0048326593823730946),
 'RightAnkle': (1.0, 0.0, 0.0, -0.0),
 'LeftHip': (0.9946348071098328,
  0.06823582202196121,
  0.05651629716157913,
  -0.05339788645505905),
 'LeftKnee': (0.9970093965530396,
  0.07583702355623245,
  0.0036906616296619177,
  0.014400972053408623),
 'LeftAnkle': (1.0, 0.0, 0.0, -4.336808689942018e-19),
 'Spine': (0.998317301273346,
  0.05599140003323555,
  0.002890974283218384,
  -0.014803871512413025),
 'Thorax': (0.9980188608169556,
  -5.0415401020575956e-17,
  0.0629153698682785,
  8.673615828920387e-19),
 'Neck': (0.9652291536331177,
  0.11822173744440079,
  0.2326054871082306,
  -0.015841426327824593),
 'LeftShoulder': (0.7188339829444885,
  0.10017464309930801,
  0.07850

In [21]:
all_rotations[-1]

{'Hip': (0.9946821331977844,
  0.09762436151504517,
  0.03197874128818512,
  0.00736644584685564),
 'RightHip': (0.9988162517547607,
  0.02245839312672615,
  -0.041628964245319366,
  -0.011346464045345783),
 'RightKnee': (0.9989937543869019,
  0.044621266424655914,
  0.004476078785955906,
  0.0005894927307963371),
 'RightAnkle': (1.0, 0.0, 0.0, -0.0),
 'LeftHip': (0.9945905804634094,
  0.05827661231160164,
  0.04028455913066864,
  -0.07596433162689209),
 'LeftKnee': (0.997927725315094,
  0.04921230673789978,
  -0.000599504099227488,
  0.041449692100286484),
 'LeftAnkle': (1.0, 0.0, 0.0, -1.734723475976807e-18),
 'Spine': (0.9995905756950378,
  0.02766968309879303,
  -0.0007996728527359664,
  0.007237588055431843),
 'Thorax': (0.9995258450508118,
  1.4248364722238213e-17,
  -0.030790304765105247,
  -1.7821571142278948e-18),
 'Neck': (0.9615914225578308,
  0.05881047248840332,
  0.26710811257362366,
  -0.023164041340351105),
 'LeftShoulder': (0.7133551239967346,
  0.14449548721313477,
  

In [22]:
from mathutils import Quaternion

In [23]:
def rodrigues_from_pose(frame, joint_name):

    quat = all_rotations[frame][joint_name]
    quat = mathutils.Quaternion(quat)
    (axis, angle) = quat.to_axis_angle()
    rodrigues = axis
    rodrigues.normalize()
    rodrigues = rodrigues * angle
    return rodrigues

In [24]:
len(all_rotations)

1269

In [25]:
joint_names = body_parts
num_joints = len(joint_names)
frame = 0
all_poses = []


for frame in range(len(all_rotations)):
    # Get armature pose in rodrigues representation
    pose = [0.0] * (num_joints * 3)

    for index in range(num_joints):
        joint_name = joint_names[index]
        joint_pose = rodrigues_from_pose(frame, joint_name)
        pose[index*3 + 0] = joint_pose[0]
        pose[index*3 + 1] = joint_pose[1]
        pose[index*3 + 2] = joint_pose[2]
    
    all_poses.append(pose)

In [26]:
np.asarray(all_poses).shape

(1269, 48)

In [27]:
print(all_poses[0])

[0.16751985251903534, -0.08717761188745499, 0.0026905930135399103, 0.10371872037649155, -0.1022740975022316, 0.015497185289859772, 0.12699757516384125, 0.0134237976744771, -0.009671890176832676, 0.0, 0.0, -0.0, 0.13671500980854034, 0.11323416978120804, -0.10698622465133667, 0.1518254429101944, 0.007388691417872906, 0.028830692172050476, 0.0, 0.0, 0.0, 0.11204594373703003, 0.005785208661109209, -0.029624436050653458, -1.0089756544275229e-16, 0.1259140521287918, 1.735871809462184e-18, 0.2392226755619049, 0.4706791341304779, -0.03205525875091553, 0.22152917087078094, 0.17360912263393402, -1.5113623142242432, -4.18194525914882e-17, -0.32703277468681335, 1.1500348966350887e-16, 0.0, 0.0, -0.0, 0.5758514404296875, 0.2348553091287613, 1.408467411994934, 8.408380923852172e-17, 0.4836369454860687, 1.121117324164725e-16, 0.0, 0.0, -0.0]


In [28]:
theta_calculated = all_poses[0]
theta_blender = [0.025758972391486168, -1.0218111276626587, -0.06451008468866348, 0.010538547299802303, 0.002836085157468915, -0.31717243790626526, -0.35010454058647156, -0.08993297815322876, -0.3302212059497833, 0.03430446237325668, 0.0013791139936074615, -0.058626383543014526, 0.5688444972038269, 0.015493002720177174, 0.46614518761634827, 0.6609958410263062, 0.08662189543247223, 0.5446861982345581, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 8.78229613476833e-08, 0.28811395168304443, -1.0198794342386464e-07, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.28448763489723206, -0.4826003909111023, -0.1608278602361679, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.5094163417816162, -0.7409713268280029, -1.0656542778015137, 0.35518118739128113, 0.5174011588096619, 0.9821890592575073, 2.300339758676273e-07, -1.575303077697754, 3.0671202466692193e-07, -6.761420934253692e-08, 1.5096821784973145, -6.986807647990645e-07, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0]

In [29]:
# import numpy as np

# def reorder_and_fill_with_mapping(A, B, group_size=3):
#     """
#     Reorder B to match A's order and add missing groups from A to B.
#     Also output a mapping for reordering and padding.
#     """
#     def normalize(value):
#         return 0 if np.isclose(value, 0, atol=1e-6) else value

#     # Normalize A and B
#     normalized_A = [normalize(a) for a in A]
#     normalized_B = [normalize(b) for b in B]

#     # Divide B into groups of group_size
#     grouped_B = [normalized_B[i:i + group_size] for i in range(0, len(normalized_B), group_size)]

#     # Create output groups and the mapping
#     output_groups = []
#     mapping = []

#     # Iterate over A in groups
#     for i in range(0, len(normalized_A), group_size):
#         group_A = normalized_A[i:i + group_size]
#         found = False

#         for j, group_B in enumerate(grouped_B):
#             if group_B is not None and all(np.isclose(a, b, atol=1e-6) for a, b in zip(group_A, group_B)):
#                 output_groups.append(group_B)
#                 mapping.append((j, None))  # Use index j of B, no padding
#                 grouped_B[j] = None  # Mark as used
#                 found = True
#                 break

#         if not found:
#             output_groups.append(group_A)
#             mapping.append((None, group_A))  # No corresponding index in B, use A's group for padding

#     # Flatten the output groups to form reordered and filled B
#     reordered_B = [item for group in output_groups for item in group]
#     return reordered_B, mapping


def apply_mapping(B, mapping, group_size=3):
    """
    Apply a mapping to reorder and pad a list like B.
    """
    # Divide B into groups of group_size
    grouped_B = [B[i:i + group_size] for i in range(0, len(B), group_size)]

    # Apply mapping
    output_groups = []
    for index, padding in mapping:
        if index is not None:
            output_groups.append(grouped_B[index])  # Reorder from B
        elif padding is not None:
            output_groups.append(padding)  # Add padding group

    # Flatten the output groups
    reordered_B = [item for group in output_groups for item in group]
    return reordered_B



# # Get reordered B and the mapping
# reordered_B, mapping = reorder_and_fill_with_mapping(theta_blender, theta_calculated, group_size=3)
# print("Reordered and filled B:", reordered_B)
# print("Mapping:", mapping)



# applied_B = apply_mapping(theta_calculated, mapping, group_size=3)
# print("Applied B:", applied_B)


In [30]:
#TODO
# Manage to get all pose parameters in the correct order so:
# The above code takes the calculated thetas and does it for one pose. This should be done for all poses
# then visualize the code usinf Ubuntu mmlab code
#U can alos use that visualization code to get the vertices from the SMPL parameters

In [31]:
theta_blender = theta_blender = [0.025758972391486168, -1.0218111276626587, -0.06451008468866348, 0.010538547299802303, 0.002836085157468915, -0.31717243790626526, -0.35010454058647156, -0.08993297815322876, -0.3302212059497833, 0.03430446237325668, 0.0013791139936074615, -0.058626383543014526, 0.5688444972038269, 0.015493002720177174, 0.46614518761634827, 0.6609958410263062, 0.08662189543247223, 0.5446861982345581, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 8.78229613476833e-08, 0.28811395168304443, -1.0198794342386464e-07, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.28448763489723206, -0.4826003909111023, -0.1608278602361679, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.5094163417816162, -0.7409713268280029, -1.0656542778015137, 0.35518118739128113, 0.5174011588096619, 0.9821890592575073, 2.300339758676273e-07, -1.575303077697754, 3.0671202466692193e-07, -6.761420934253692e-08, 1.5096821784973145, -6.986807647990645e-07, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0]
all_thetas = []

for i, frame in enumerate(all_poses):
    theta_calculated = all_poses[i]

    reordered_theta = apply_mapping(theta_calculated, mapping)
    smpl_h_params = reordered_theta

    reordered_theta = reordered_theta[0:72]

    all_thetas.append(reordered_theta)

    
    

In [32]:
len(all_thetas)

1269

In [33]:
all_thetas[0]

[0.16751985251903534,
 -0.08717761188745499,
 0.0026905930135399103,
 0.13671500980854034,
 0.11323416978120804,
 -0.10698622465133667,
 0.10371872037649155,
 -0.1022740975022316,
 0.015497185289859772,
 0.11204594373703003,
 0.005785208661109209,
 -0.029624436050653458,
 0.1518254429101944,
 0.007388691417872906,
 0.028830692172050476,
 0.12699757516384125,
 0.0134237976744771,
 -0.009671890176832676,
 0.0,
 0.0,
 -0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0,
 -1.0089756544275229e-16,
 0.1259140521287918,
 1.735871809462184e-18,
 0.0,
 0.0,
 -0.0,
 0,
 0,
 0,
 0.2392226755619049,
 0.4706791341304779,
 -0.03205525875091553,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.22152917087078094,
 0.17360912263393402,
 -1.5113623142242432,
 0.5758514404296875,
 0.2348553091287613,
 1.408467411994934,
 -4.18194525914882e-17,
 -0.32703277468681335,
 1.1500348966350887e-16,
 8.408380923852172e-17,
 0.4836369454860687,
 1.121117324164725e-16,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [34]:
all_thetas = np.asarray(all_thetas)

In [35]:
all_thetas.shape

(1269, 72)

In [36]:
np.save("all_thetas_backpack.npy", all_thetas)

In [70]:


# TODO Get the parameters into the correct order

In [1]:
import pickle as pkl

data_gt = pkl.load(open(r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\3DPW\sequenceFiles\sequenceFiles\test\flat_guitar_01.pkl", 'rb'), encoding='latin1')


In [4]:
import numpy as np

In [6]:
np.max(data_gt["jointPositions"])

1.027466730454574

In [49]:
path = r"E:\Users\Philipp\Dokumente\Pose_Estimation_3D\MotionBert\MotionBERT\data\mesh\J_regressor_h36m_correct.npy"

In [50]:
import numpy as np

# Load the regressor
regressor = np.load(path)

# Check the shape of the regressor
print("Regressor shape:", regressor.shape)

# Optionally, inspect the contents (prints part of the array)
print("Regressor contents:", regressor)


Regressor shape: (17, 6890)
Regressor contents: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def rotate_bone(initial_euler, rotation_axis, rotation_angle):
    # Step 1: Convert initial Euler angles to a rotation matrix (Blender uses degrees)
    initial_rotation = R.from_euler('zxy', initial_euler, degrees=True)
    
    # Step 2: Create the rotation matrix for the global rotation around the X-axis (-90 degrees)
    rotation_matrix = R.from_euler(rotation_axis, rotation_angle, degrees=True)
    
    # Step 3: Multiply the rotation matrices (the new matrix is applied globally)
    resulting_rotation = rotation_matrix * initial_rotation
    
    # Step 4: Convert the resulting rotation matrix back to Euler angles
    result_euler = resulting_rotation.as_euler('zxy', degrees=True)
    
    return result_euler

# Initial orientation (Z, X, Y) = (31.4473, -89.4163, -93.4718)
initial_euler = [31.4473, -89.4163, +93.4718]

# Rotate globally around the X-axis by -90 degrees
rotation_axis = 'x'
rotation_angle = -90

# Get the new Euler angles
new_euler = rotate_bone(initial_euler, rotation_axis, rotation_angle)
print(new_euler)


[-5.50810797e+01  3.53466678e-02  1.79417371e+02]
